<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_synthesis/blob/main/sound_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle

import librosa
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class Loader:
  def __init__(self,sample_rate,duration,mono): #저장된 파일 읽어오기
    self.sample_rate=sample_rate #현재 클래스의 내용으로 저장
    self.duration=duration
    self.mono=mono #단일 여부 저장

  def load(self,file_path): #주어진 위치의 파일에서 시그널로 먼저 읽어옴
    signal=librosa.load(file_path, #파일은 E:\프로그래밍 공부용 파일에 있음, archive
                        sr=self.sample_rate,
                        duration=self.duration,
                        mono=self.mono)[0]
    return signal

class Padder:
  def __init__(self,mode="constant"):
    self.mode=mode

  def left_pad(self,array,num_missing_items): #넘파이 배열로 바뀐 스펙트럼에 패딩 추가, 앞쪽
    padded_array=np.pad(array,
                        (num_missing_items,0),#num_missing은 총 길이에서 현재 샘플길이만큼 뺀값
                        mode=self.mode)
    return padded_array

  def right_pad(self,array,num_missing_items):# 뒤쪽에 패딩 추가
    padded_array=np.pad(array,
                        (0,num_missing_items),
                        mode=self.mode)
    return padded_array

class LogSpectrogramExtractor: #로그 스펙트럼으로 추출
  def __init__(self,frame_size,hop_length): #로그 스펙트럼 기본 설정
    self.frame_size=frame_size #각 프레임 크기
    self.hop_length=hop_length # 각 프레임별로 얼마나 움직일지 설정

  def extract(self,signal): #시그널 추출
    stft=librosa.stft(signal,#단시간 푸리에 변환
                      n_fft=self.frame_size,
                      hop_lenghth=self.hop_length)[:-1]
    spectrogram=np.abs(stft) #푸리에 변환 절댓값
    log_spectrogram=librosa.amplitude_to_db(spectrogram) #진폭을 데시벨로 변환
    return log_spectrogram

class MinMaxNormaliser: #정규화(?)
  def __init__(self,min_val,max_val):
    self.min=min_val#정규화 최소치
    self.max=max_val#정규화 최고치

  def normalise(self,array): #0부터 1까지 정규화
    norm_array=(array-array.min())/(array.max()-array.min())
    norm_array=norm_array*(self.max-self.min)+self.min
    return norm_array

  def denormalise(self,norm_array,original_min, original_max):
    array=(norm_array-self.min)/(self.max-self.min)
    array=array*(original_max-original_min)+original_min
    return array

class Saver: #저장기능(저장 위치)
  def __init__(self,feature_save_dir,min_max_values_save_dir):
    self.feature_save_dir=feature_save_dir
    self.min_max_values_save_dir=min_max_values_save_dir

  def save_feature(self,feature,file_path):#요소 저장
    save_path=self._generate_save_path(file_path)
    np.save(save_path,feature)

  def save_min_max_values(self,min_max_values):#최댓값, 최솟값 저장
    save_path=os.path.join(self.min_max_values_save_dir,
                           "min_max_values.pkl")
    self._save(min_max_values, save_path)

  @staticmethod
  def _save(data, save_path): #데이터 저장
      with open(save_path, "wb") as f:
          pickle.dump(data, f)


class PreprocessingPipeline: #파이프라인 선언
  """
  1. 파일 로드
  2. 파일의 파형 패딩(길이 일치시키기)
  3. 로그 스펙트로그램 추출
  4. 스펙트로그램 정규화
  5. 정규화 값 저장
  """
  def __init__(self): #프로세스 순서 선언
    self.padder=None
    self.extractor=None
    self.normaliser=None
    self.saver=None
    self.min_max_values={}
    self._loader=None
    self._num_expected_samples=None

  def loader(self): #원 데이터 로딩
    return self._loader

  def loader(self,loader): #다른 파일에서 가져오기(?)
    self._loader=loader
    self._num_expected_samples=int(loader.sample_rate * loader.duration)

  def process(self,audio_files_dir): #프로세싱, 각 파일에 대해서 실행
    for root,_,files in os.walk(audio_files_dir):#"_"는 디렉토리 이름인데 필요없음
      for file in files:
        file_path=os.path.join(root,file)
        self._process_file(file_path)
        print(f"Processed file {file_path}")
    self.saver.save_min_max_values(self.min_max_values)

  def _process_file(self,file_path): #하나씩 골라진 파일에 대해서 실행
    signal=self.loader.load(file_path) #로드
    if self._is_padding_necessary(signal): #각 파일에 대해서 패딩필요여부 확인, 길이가 충분한가?
      signal=self._apply_padding(signal) #충분하지 않으면 패딩으로 길이 늘림
    feature=self.extractor.extract(signal) #요소 추출
    norm_feature=self.normaliser.normalise(feature)# 정규화(?)
    save_path=self.saver.save_feature(norm_feature,file_path)#프로세싱 끝난 내용 저장
    self._store_min_max_value(save_path,feature.min(),feature.max())# 최솟값, 최댓값 저장

  def _is_padding_necessary(self,signal):#모자란 길이에 대해서 패딩여부 조사
    if len(signal)<self._num_expected_samples:
      return True
    return False

  def _apply_padding(self,signal): #앞쪽에 패딩 추가
    num_missing_samples=self._num_expected_samples-len(signal)
    padding_signal=self.padder.right_pad(signal,num_missing_samples)
    return padded_signal

  def _store_min_max_value(self,save_path,min_val,max_val):#최소 최대 저장
    self.min_max_values[save_path]={
        "min":min_val,
        "max":max_val
    }

if __name__=="__main__":
  FRAME_SIZE=512 #512개로 샘플 나눔
  HOP_LENGTH=256
  DURATION=0.74 #지정된 최대 샘플길이
  SAMPLE_RATE=22050
  MONO=True

  SPECTROGRAMS_SAVE_DIR="/content/drive/MyDrive/오디오_합성/"
  MIN_MAX_VALUES_SAVE_DIR="/content/drive/MyDrive/오디오_합성/"
  FILES_DIR="/content/drive/MyDrive/오디오_합성/"

  loader=Loader(SAMPLE_RATE,DURATION,MONO)
  padder=Padder()
  log_spectrogram_extractor=LogSpectrogramExtractor(FRAME_SIZE,HOP_LENGTH)
  min_max_normaliser=MinMaxNormaliser(0,1)
  saver=Saver(SPECTROGRAMS_SAVE_DIR,MIN_MAX_VALUES_SAVE_DIR)

  preprocessing_pipeline=PreprocessingPipeline()
  preprocessing_pipeline.loader=loader
  preprocessing_pipeline.padder=padder
  preprocessing_pipeline.extractor=log_spectrogram_extractor
  preprocessing_pipeline.normaliser=min_max_normaliser
  preprocessing_pipeline.saver=saver

  preprocessing_pipeline.process(FILES_DIR)

<ipython-input-15-8768ef54f91e>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  signal=librosa.load(file_path,
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


NoBackendError: 

**스펙트로그램을 역 푸리에변환을 통해 파형으로 변환함**



